# PyProj

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import sys
import os
import cx_Oracle
from pprint import pprint

получаем информацию о компаниях, перечисленных в excel

In [53]:
def get_companies():
    tables = pd.read_html("https://www.nalog.ru/opendata/7707329152-reestropereldoc/", encoding="UTF-8", header=0,
                          index_col="№")
    frame = tables[0]
    registry = frame['Значение характеристики'].loc[8]  # url реестра

    url_validator(registry)

    company_list = pd.read_excel('C:\\code\\test.xlsx')['Компания'].values.tolist()
    #print("содержимое excel файла")
    #print(company_list)
    orders = pd.read_csv(registry, sep=';', header=0, usecols=['GB', 'G1', 'G2', 'G4'])
    """GA	Порядковый номер
       GB	Наименование организации
       G1	Адрес компании
       G2	ИНН организации
       G3	Номер договора
       G4	Интернет сайт
       G5	Наличие автоустановщика для настроки програмного обеспечения при работе с электронной подписью
       """
    comp_names = orders["GB"].str.lower()
    matching = comp_names.isin([x.lower() for x in company_list])
    for cn in company_list:
        if cn.lower() not in list(comp_names):
                print("Нет данных по {}".format(cn.lower()))
    return orders[matching]

In [54]:
def url_validator(url):
    result = urlparse(url)
    nl = r"data.nalog.ru"
    f_format = ".csv"
    if not all([result.netloc == nl, result.path.endswith(f_format)]):
        print("некорректная ссылка на файл " + url)
        sys.exit(0)
    else:
        return

Создаём связь с базой данных

In [55]:
cx_Oracle.init_oracle_client(lib_dir="C:\\code\\instantclient_19_9forever")

ProgrammingError: Oracle Client library has already been initialized

Записываем строки в таблицу


In [56]:
    DF = get_companies()
    DF.index = np.arange(len(DF))
    #n = DF.loc[pd.isna(DF['G5']), 'G5'] 
    if len(DF) == 0:
        print("нет совпадений")
        sys.exit(0)
    connectionString = '(description =(retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=lpdmasisaukuwfd_db202009tomch_high.adwc.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adwc.eucom-central-1.oraclecloud.com,OU=Oracle BMCS FRANKFURT,O=Oracle Corporation,L=Redwood City,ST=California,C=US")))'
    conn = cx_Oracle.connect(r'ADMIN', r'M@i806M@i806', dsn=connectionString)
    cursor = conn.cursor()
    #print(DF)
    for c in range(len(DF)):
        if not pd.isna(DF['GB'][c]):
            try:
                cursor.execute("""begin 
                                UPDATE REESTROPERELDOC SET ADDRESS = :ADDRESS, TIN = :TIN, WEBSITE = :WEBSITE 
                                WHERE ORG_NAME = :ORG_NAME;
                               IF sql%notfound
                               THEN INSERT INTO REESTROPERELDOC(ORG_NAME, ADDRESS, TIN, WEBSITE) 
                               VALUES(:ORG_NAME, :ADDRESS, :TIN, :WEBSITE);
                               END IF;
                               end;""", 
                                {"ADDRESS": str(DF['G1'][c]), 
                                "TIN": str(DF['G2'][c]), 
                                "WEBSITE": str(DF['G4'][c]), 
                                "ORG_NAME": str(DF['GB'][c])})
            except cx_Oracle.DatabaseError as e:
                print ("Ошибка: ", e)
            else:
                conn.commit()
    conn.close()

Нет данных по ооо хаха


In [60]:
conn = cx_Oracle.connect(r'ADMIN', r'M@i806M@i806', dsn=connectionString) 
#cursor.execute("SELECT * FROM REESTROPERELDOC")
#result = cursor.fetchall()
result = pd.read_sql("""SELECT * FROM REESTROPERELDOC""", conn)
print(result)
conn.close()

          inn
0  6663003127
1  4029017981
2  6454066437
3  1831014533
4  6608003052
5          MM


In [43]:
connectionString = '(description =(retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.eu-frankfurt-1.oraclecloud.com))(connect_data=(service_name=lpdmasisaukuwfd_db202009tomch_high.adwc.oraclecloud.com))(security=(ssl_server_cert_dn="CN=adwc.eucom-central-1.oraclecloud.com,OU=Oracle BMCS FRANKFURT,O=Oracle Corporation,L=Redwood City,ST=California,C=US")))'
   
conn = cx_Oracle.connect(r'ADMIN', r'M@i806M@i806', dsn=connectionString) 
cursor = conn.cursor()
cursor.execute("SELECT * FROM v$version")
result = cursor.fetchall()
print(result)
conn.close()

[('Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production', 'Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production\nVersion 19.5.0.0.0', 'Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production', 0)]
